## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. (необязательное) Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

from scipy import stats

np.set_printoptions(threshold=np.nan)

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0  # ???
    LEAF_TYPE = 1  # ???

    def __init__(self, min_samples_split=2, max_depth=None, criterion='gini', max_features=None):
        # sufficient_share=1.0
        """
        min_samples_split: 
            Минимальное количество выборок, необходимое для разделения внутреннего узла
        max_depth:
            Максимальная глубина дерева. Если None, то узлы расширяются до тех пор, 
            пока все листья не станут чистыми или пока все листья не будут содержать меньше 
            чем min_samples_split значений.
        criterion:
            функция энтропии
        max_features:
            Количество фич, которые следует учитывать при поиске лучшего разделения
                Если None, то max_features = n_features
                Если int, то учитывайть фичи max_features при каждом разделении
                Если "sqrt", то max_features=sqrt(n_features).
                Если "log2", то max_features=log2(n_features).
        """
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        #self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    # left_class_count, left_sizes, right_class_count, right_sizes
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        gini_l = np.sum(l_c*l_c, axis=1,keepdims=1)
        gini_r = np.sum(r_c*r_c, axis=1,keepdims=1)
        return (gini_l/l_s + gini_r/r_s)/(l_s + r_s)

    def __entropy(self, l_c, l_s, r_c, r_s):
        p_l=l_c/l_s
        p_r=r_c/r_s
        th=10e-8
        e_l=-np.nansum(p_l*np.log2(np.clip(p_l, th, 1 - th)), axis=1,keepdims=1)
        e_r=-np.nansum(p_r*np.log2(np.clip(p_r, th, 1 - th)), axis=1,keepdims=1)
        return  1-(l_s*e_l+r_s*e_r)/(l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        mc_l = 1-np.max(l_c/l_s,axis=1,keepdims=1)
        mc_r = 1-np.max(r_c/r_s,axis=1,keepdims=1)
        return  1-(l_s*mc_l+r_s*mc_r)/(l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature,dtype=np.int)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature)+0.5)]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature,dtype=np.int)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature)+0.5)]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        """
        ищет лучшее разбиение только по одному признаку, 
        поэтому в качестве агрумента x должна быть не все матрица признаков, 
        а только один ее столбец.
        """

        # Что делает этот блок кода?
        # сортирует фичу по возрастанию с сохранением номеров класса
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # количество уникальных классов на текущей иттерации
        class_number = np.unique(y).shape[0]

        # Что делает этот блок кода?
        # определяем границы там, где меняется класс, сохраняя self.min_samples_split
        splitted_sorted_y = sorted_y[self.min_samples_split:-
                                     self.min_samples_split]
        r_border_ids = np.where(
            splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)

        if len(r_border_ids) == 0:
#             return float('+inf'), None
            return float('-inf'), None
            

        # Что делает этот блок кода?
        # определяет сколько и каких классов останется
        # при каждом разбиении по границам из r_border_ids
        eq_el_count = r_border_ids - \
            np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:self.min_samples_split],
                        minlength=class_number)

        # Что делает этот блок кода?
        # число классов в левой вершине после разбиения
        l_class_count = np.cumsum(class_increments, axis=0)
        # справа
        r_class_count = np.bincount(y) - l_class_count
        # общее число объектов слева
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        # справа
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # считает прирост информации и ищет аргумент минимального значения
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
#         idx = np.argmin(gs)
        idx = np.argmax(gs) 

        # Что делает этот блок кода?
        # Возвращает минимальное значение меры неопределенности и порог при этом значении
        left_el_id = l_sizes[idx][0]
        gs_best = gs[idx]
        # если случай [0,0,0,1,1,1]
        #             [0,1,1,1,1,1]
        # то пересчитываем порог и прирост информации(дорого по скорости ~x3)
        if sorted_x[left_el_id-1] == sorted_x[left_el_id]:
            delta = np.min(sorted_x[1:] - sorted_x[:-1])/2.0
            th = sorted_x[left_el_id-1] + delta
            x_s = x.reshape(x.shape[0],1)
            x_l, x_r, y_l, y_r = self.__div_samples(x_s, y, 0, th)
            l_class_count = np.bincount(y_l)
            l_class_count = l_class_count.reshape(1,l_class_count.size)
            r_class_count = np.bincount(y_l)
            r_class_count = r_class_count.reshape(1,r_class_count.size)
            l_sizes = np.array([[y_l.size]])
            r_sizes = np.array([[y_r.size]])
            gs_best = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        else:
            th = (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
        return gs_best, th

    def __fit_node(self, x, y, node_id, depth, pred_cls=None, pred_p=None):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
#         i_g_best = 1
        i_g_best = 0        
        feature_id_best = -1
        th_best = 0
        xT=x.T
        # ищем лучший порог разбиения
        for i in self.get_feature_ids(xT.shape[0]):
            x_f=xT[i,:]
            if np.unique(x_f).size == 1:
                continue
            y_u = np.unique(y)
            y_m = np.zeros(y.size, dtype=np.int)
            for j in range(1, y_u.size):
#                 print((y == y_u[j]).shape,y_m.shape)
                y_m[y == y_u[j]] = j
            i_g, th = self.__find_threshold(x_f, y_m)
#             print(i_g,i,th)
#             if i_g < i_g_best:
            if th is None:
                continue
            if i_g > i_g_best:
                i_g_best = i_g
                th_best = th
                feature_id_best = i
        # если порог не найден, значит, лист дерева
        if feature_id_best == -1:
#             print("no th")
            mode = stats.mode(y)
            y_cls, y_num = mode[0][0], mode[1][0]
            self.tree[node_id] = (self.__class__.LEAF_TYPE,
                                  y_cls, (y_cls,y_num/y.size))
            return
#         print(node_id, depth, i_g_best, feature_id_best, th_best)
        # иначе делим выборку в дереве
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_id_best, th_best)
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE,
                              feature_id_best, th_best)
#         print(x_l.shape,x_r.shape)
#         if x_l.shape[0]==0 or x_r.shape[0]==0:
#             print(x_l[:,feature_id_best],x_r[:,feature_id_best])

        is_max_depth = False
        if self.max_depth:
            if self.max_depth >= depth:
                is_max_depth = True
        mode = stats.mode(y_l)
        y_l_cls, y_l_num = mode[0][0], mode[1][0]
        mode = stats.mode(y_r)
        y_r_cls, y_r_num = mode[0][0], mode[1][0]

        # достигнута макс. глубина или идеально разделили
        if is_max_depth or i_g_best == 0.0:
            self.tree[2*node_id+1] = (self.__class__.LEAF_TYPE,
                                      y_l_cls, (y_l_cls, y_l_num/y_l.size))
            self.tree[2*node_id+2] = (self.__class__.LEAF_TYPE,
                                      y_r_cls, (y_r_cls, y_l_num/y_l.size))
#             print("stop")
        # если min_samples_split или отсеяли одну часть слева
        elif x_l.shape[0] <= self.min_samples_split*2 or np.unique(y_l).size == 1:
#             print("continue left")
            self.tree[2*node_id+1] = (self.__class__.LEAF_TYPE,
                                      y_l_cls, (y_l_cls, y_l_num/y_l.size))
            # а вдруг справа?
            if x_l.shape[0] <= self.min_samples_split*2:
                self.tree[2*node_id+2] = (self.__class__.LEAF_TYPE,
                                          y_r_cls, (y_r_cls, y_l_num/y_l.size))
            else:
                self.__fit_node(x_r, y_r, 2*node_id+2, depth+1,
                                y_r_cls, (y_r_cls, y_l_num/y_l.size))
        # если min_samples_split или отсеяли одну часть справа
        elif x_r.shape[0] <= self.min_samples_split*2 or np.unique(y_r).size == 1:
#             print("continue right")
            self.tree[2*node_id+2] = (self.__class__.LEAF_TYPE,
                                      y_r_cls, (y_r_cls, y_l_num/y_l.size))
            if x_r.shape[0] <= self.min_samples_split*2: # для симметрии =)
                self.tree[2*node_id+1] = (self.__class__.LEAF_TYPE,
                                          y_l_cls, (y_l_cls, y_l_num/y_l.size))
            else:
                self.__fit_node(x_l, y_l, 2*node_id+1, depth+1,
                                y_r_cls, (y_r_cls, y_l_num/y_l.size))
        # в противном случае, делим дерево
        else:
#             print("continue bouth")
            self.__fit_node(x_l, y_l, 2*node_id+1, depth+1,
                            y_l_cls, (y_l_cls, y_l_num/y_l.size))
            self.__fit_node(x_r, y_r, 2*node_id+2, depth+1,
                            y_r_cls, (y_r_cls, y_l_num/y_l.size))
        return

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


Проверим на синтетических идеальных данных

In [3]:
np.random.seed(0)
# x = np.array(5*np.random.rand(500,1),dtype=np.int32)
x = np.array(5*np.random.rand(500, 2), dtype=np.int32)
y = np.zeros(x.shape[0], dtype=np.int)
y[np.where(x[:, 0] > 2)[0]] = 1
# y[np.where(x[:, 0] > 3)[0]] = 2
y[np.where(x[:, 1] > 3)[0]] = 2
y[np.where((x[:, 1] > 3) & (x[:, 0] > 2))[0]] = 3
print(x.shape, y.shape)
print(np.c_[x,y][:10,:])
m_clf = MyDecisionTreeClassifier()
m_clf.fit(x, y)
y_pred=m_clf.predict(x)
print(np.c_[y,y_pred][:10,:])
print('accuracy score',accuracy_score(y_pred=y_pred, y_true=y))

(500, 2) (500,)
[[2 3 0]
 [3 2 1]
 [2 3 0]
 [2 4 2]
 [4 1 1]
 [3 2 1]
 [2 4 2]
 [0 0 0]
 [0 4 2]
 [3 4 3]]
[[0 0]
 [1 1]
 [0 0]
 [2 2]
 [1 1]
 [1 1]
 [2 2]
 [0 0]
 [2 2]
 [3 3]]
accuracy score 1.0


E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide
E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:202: RuntimeWarning: invalid value encountered in greater


In [4]:
df = pd.read_csv('./cs_training.csv', sep=',').dropna()
df.shape

(120269, 11)

In [5]:
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [6]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [7]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [8]:
# %prun my_clf.fit(x, y)

In [9]:
# from pycallgraph import PyCallGraph
# from pycallgraph.output import GraphvizOutput

# with PyCallGraph(output=GraphvizOutput()):
#     my_clf.fit(x, y)

In [10]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide
E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:202: RuntimeWarning: invalid value encountered in greater


18.27403163909912
1.8735179901123047


Если убрать эвристику в __find_threshold, то считает в 3-4 раза быстрее, но все равно раза в 3 медленнее стандартной реализации. Нужно переписывать эту функцию. В противном случае не работает на реальных данных.

## Проверка качества работы

In [11]:
gkf = KFold(n_splits=5, shuffle=True)

In [12]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide
E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:202: RuntimeWarning: invalid value encountered in greater


0.9165211607217095
0.9161054294504033
0.9195559990022449
0.9158975638147502
0.9138568993472748


In [13]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.891660430697597
0.8921593082231646
0.8901637981208946
0.892325600731687
0.8921548247619839


# Применить для задачи Titanic 

In [14]:
df = pd.read_csv('./train.csv', sep=',')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
"""
PassengerId:type should be integers
Survived:Survived or Not
Pclass:Class of Travel
Name:Name of Passenger
Sex:Gender
Age:Age of Passengers
SibSp:Number of Sibling/Spouse aboard
Parch:Number of Parent/Child aboard
Ticket
Fare
Cabin
Embarked:The port in which a passenger has embarked. C - Cherbourg, S - Southampton, Q = Queenstown
"""

'\nPassengerId:type should be integers\nSurvived:Survived or Not\nPclass:Class of Travel\nName:Name of Passenger\nSex:Gender\nAge:Age of Passengers\nSibSp:Number of Sibling/Spouse aboard\nParch:Number of Parent/Child aboard\nTicket\nFare\nCabin\nEmbarked:The port in which a passenger has embarked. C - Cherbourg, S - Southampton, Q = Queenstown\n'

In [16]:
df = df.drop(columns=['PassengerId','Name','SibSp','Ticket','Fare','Cabin','Embarked'])
print(df.shape)
df = df.dropna()
print(df.shape)

(891, 5)
(714, 5)


In [17]:
df.head()

,Survived,Pclass,Sex,Age,Parch
0,0,3,male,22.0,0
1,1,1,female,38.0,0
2,1,3,female,26.0,0
3,1,1,female,35.0,0
4,0,3,male,35.0,0


In [18]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
d = defaultdict(LabelEncoder)

X = np.c_[
    df[['Sex']].apply(lambda x: d[x.name].fit_transform(x)).as_matrix(),
    df[['Pclass', 'Age','Parch']].as_matrix()
]

y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])
X.shape, y.shape

E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


((714, 4), (714,))

In [19]:
dt = MyDecisionTreeClassifier()

In [20]:
gkf = KFold(n_splits=5, shuffle=True)
for train, test in gkf.split(X, y):
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    dt.fit(X_train, y_train)
    print(accuracy_score(y_pred=dt.predict(X_test), y_true=y_test))

E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide
E:\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:202: RuntimeWarning: invalid value encountered in greater


0.8881118881118881
0.7622377622377622
0.8041958041958042
0.7692307692307693
0.795774647887324
